## ODC-geo with H5NetCDF


In [ ]:
import argparse

import fsspec
import xarray as xr
from common import earthaccess_args
from odc.geo.geobox import GeoBox
from odc.geo.xr import xr_reproject

In [ ]:
def warp_resample(dataset):
    args = earthaccess_args[dataset]
    src = f'earthaccess_data/{args["filename"]}'
    dstSRS = "EPSG:3857"
    srcSRS = "EPSG:4326"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]
    fs = fsspec.filesystem("file")
    with fs.open(src) as f:
        gbox = GeoBox.from_bbox(te, dstSRS, shape=(height, width))
        da = xr.open_dataset(f, engine="h5netcdf")[args["variable"]]
        if dataset == "gpm_imerg":
            da = (
                da.rename({"lon": "x", "lat": "y"})
                .transpose("time", "y", "x")
                .squeeze()
            )
        da = da.odc.assign_crs(srcSRS)
        return xr_reproject(da, gbox)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        da = warp_resample("gpm_imerg")
    else:
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["gpm_imerg", "mursst"],
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset)